In [2]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import urllib.request as urllib2
import time
from selenium.webdriver.support.select import Select
import re
import os

### Ball by ball data (Initial)

In [56]:
#Declaring all lists and dataframes
match_list=[]
inns_list=[]
ball_list=[]
over_list=[]
runs_list=[]
comm_short_list=[]
comm_long_list=[]
bat_team_list=[]
bowl_team_list=[]
striker_bat_id_list=[]
nonstriker_bat_id_list=[]
striker_bat_name_list=[]
nonstriker_bat_name_list=[]
main_bow_id_list=[]
other_bow_id_list=[]
main_bow_name_list=[]
other_bow_name_list=[]
isBoundary_list=[]
wide_runs_list=[]
#legbye_runs_list=[]
bye_legbye_runs_list=[]
nb_runs_list=[]
batsman_runs_list=[]
extras_runs_list=[]
is_wkt_list=[]
is_bow_wkt_list=[]
wkt_data_list=[]
curr_bat_list=[]
curr_bow_list=[]
curr_inn_list=[]
match_over_list=[]

for mid in matchid_list:
    url_match='https://www.espncricinfo.com/series/8048/commentary/{}/trinbago-knight-riders-vs-st-kitts-and-nevis-patriots-1st-match-caribbean-premier-league-2019'.format(mid)
    driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url_match)
    content = driver.page_source
    soup = BeautifulSoup(content)
    team_list=[]
    for teams in soup.findAll('div',attrs={'class':'teams'}):
        for team in teams.findAll('div',attrs={'class':'team'}):
            team_list.append(team.find('span').attrs['title'])
    driver.quit()    
    print(' ')
    
    for inns in np.arange(1,3):
        url='https://hsapi.espncricinfo.com/v1/pages/match/comments?lang=en&leagueId=8048&eventId={match}&liveTest=false&period={inns}&page=1'.format(match=mid,inns=inns)
        driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
        driver.get(url)
        content = driver.page_source
        soup = BeautifulSoup(content)
        total_pages=json.loads(soup.text)['pagination']['pageCount']
        driver.quit()
        
        for page in np.arange(total_pages):
            #Code to retrieve json data for the page
            url_page='https://hsapi.espncricinfo.com/v1/pages/match/comments?lang=en&leagueId=8048&eventId={match}&liveTest=false&period={inns}&page={page}'.format(match=mid,inns=inns,page=page+1)
            driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
            driver.get(url_page)
            content = driver.page_source
            soup = BeautifulSoup(content)
            soup_data=json.loads(soup.text)
            print('Extracting_{match}_Inns{inns}_Page{page} content'.format(match=mid,inns=inns,page=page+1))
            #with open('Downloaded webpages\CPL_2019_{match}_Inns{inns}_Page{page}.html'.format(match=mid,inns=inns,page=page+1), 'w') as f:
            #    f.write(content)
            #driver.quit()
            #print('Downloaded CPL_2019_{match}_Inns{inns}_Page{page} webpage'.format(match=mid,inns=inns,page=page+1))
            
            #Code to load page json data into dataframe
            #with open('Downloaded webpages\CPL_2019_{match}_Inns{inns}_Page{page}.html'.format(match=mid,inns=inns,page=page+1)) as fp:
            #    soup_data = json.loads(BeautifulSoup(fp, "html5lib").text)
            for ball_ind in np.arange(len(soup_data['comments'])):
                ball_data=soup_data['comments'][ball_ind]
                match_list.append(mid)
                inns_list.append(inns)
                if inns==1:
                    bat_team_list.append(team_list[0])
                    bowl_team_list.append(team_list[1])
                else:
                    bat_team_list.append(team_list[1])
                    bowl_team_list.append(team_list[0])        
                ball_list.append(ball_data['ball'])
                over_list.append(ball_data['over']+1)
                runs_list.append(ball_data['runs'])
                comm_short_list.append(ball_data['shortText'])
                comm_long_list.append(ball_data['text'])
                striker_bat_id_list.append(ball_data['currentBatsmen'][0]['id'])
                nonstriker_bat_id_list.append(ball_data['currentBatsmen'][1]['id'])
                striker_bat_name_list.append(ball_data['currentBatsmen'][0]['name'])
                nonstriker_bat_name_list.append(ball_data['currentBatsmen'][1]['name'])
                main_bow_id_list.append(ball_data['currentBowlers'][0]['id'])
                main_bow_name_list.append(ball_data['currentBowlers'][0]['name'])
                extras_runs_list.append(0)
    
                if 'id' in ball_data['currentBowlers'][1]:
                    other_bow_id_list.append(ball_data['currentBowlers'][1]['id'])
                else:
                    other_bow_id_list.append(np.nan)

                if 'name' in ball_data['currentBowlers'][1]:
                    other_bow_name_list.append(ball_data['currentBowlers'][1]['name'])                    
                else:
                    other_bow_name_list.append(np.nan)

                if str(ball_data['isBoundary'])=='True':
                    isBoundary_list.append(1)
                else:
                    isBoundary_list.append(0)

                if str(ball_data['isWide'])=='True':
                    wide_runs_list.append(ball_data['runs'])
                    nb_runs_list.append(0)
                    bye_legbye_runs_list.append(0)
                    batsman_runs_list.append(0)
                elif ((str(ball_data['isNoball'])=='True')):
                    wide_runs_list.append(0)
                    bye_legbye_runs_list.append(0)
                    if (re.search('[(]no ball[)]', ball_data['shortText'])):
                        nb_runs_list.append(1)
                        batsman_runs_list.append(ball_data['runs'] - 1)
                    else:    
                        nb_runs_list.append(ball_data['runs'])
                        batsman_runs_list.append(0)                        
                elif ('bye' in ball_data['shortText'].split(', ')[1]):
                    bye_legbye_runs_list.append(ball_data['runs'])
                    wide_runs_list.append(0)
                    nb_runs_list.append(0)                    
                    batsman_runs_list.append(0)
                else:
                    wide_runs_list.append(0)
                    nb_runs_list.append(0)
                    batsman_runs_list.append(ball_data['runs']) 
                    bye_legbye_runs_list.append(0)
                    
                if 'matchWicket' in ball_data:
                    is_wkt_list.append(1)
                    wkt_data_list.append(ball_data['matchWicket'])
                    if re.search('(run out|obstructing the field)',ball_data['matchWicket']['text']):
                        is_bow_wkt_list.append(0)
                    else:
                        is_bow_wkt_list.append(1)
                else:
                    is_wkt_list.append(0)
                    wkt_data_list.append(0)
                    is_bow_wkt_list.append(0)
                    
                if 'matchOver' in ball_data:
                    match_over_list.append(ball_data['matchOver'])
                else:
                    match_over_list.append(0)
                    
                curr_bat_list.append(ball_data['currentBatsmen'])
                curr_bow_list.append(ball_data['currentBowlers'])
                curr_inn_list.append(ball_data['currentInning'])
            driver.quit()        
    
ball_by_ball_df=pd.DataFrame(list(zip(match_list,inns_list,over_list,ball_list,bat_team_list,bowl_team_list,striker_bat_id_list,striker_bat_name_list,nonstriker_bat_id_list,nonstriker_bat_name_list,main_bow_id_list,main_bow_name_list,other_bow_id_list,other_bow_name_list,isBoundary_list,wide_runs_list,nb_runs_list,bye_legbye_runs_list,extras_runs_list,batsman_runs_list,runs_list,is_wkt_list,is_bow_wkt_list,wkt_data_list,match_over_list,curr_bat_list,curr_bow_list,curr_inn_list,comm_short_list,comm_long_list)),
                             columns=['Match id','Inns','Over','Ball','Batting team','Bowling team','Strike batsman id','Strike batsman name','Non-strike batsman id','Non-strike batsman name','Main bowler id','Main bowler name','Other end bowler id','Other end bowler name','Is Boundary','Wide runs','No ball runs','Bye and leg bye runs','Extras runs','Batsman runs','Total Runs','Is wicket','Is Bowler wicket','Wicket data','Over data','Batsmen data','Bowlers data','Innings data until then','Short comm','Long Comm'])
ball_by_ball_df['Extras runs']=ball_by_ball_df['Wide runs']+ball_by_ball_df['No ball runs']+ball_by_ball_df['Bye and leg bye runs']
ball_by_ball_df=ball_by_ball_df.sort_values(['Match id','Inns','Over','Ball'])
ball_by_ball_df.to_csv('Output folder\BBB_data.csv',index=False) 

print('Scraping process over')

 
Extracting_1114856_Inns1_Page1 content
Extracting_1114856_Inns1_Page2 content
Extracting_1114856_Inns1_Page3 content
Extracting_1114856_Inns1_Page4 content
Extracting_1114856_Inns1_Page5 content
Extracting_1114856_Inns2_Page1 content
Extracting_1114856_Inns2_Page2 content
Extracting_1114856_Inns2_Page3 content
Extracting_1114856_Inns2_Page4 content
Extracting_1114856_Inns2_Page5 content
Scraping process over


### Matches data (Initial)

In [6]:
match_list=[]
leagueName=[]
leagueAbbreviation=[]
matchYear=[]
season=[]
matchNum=[]
date=[]
venue=[]
city=[]
country=[]
homeTeamName=[]
homeTeamAbbr=[]
homeTeamCap=[]
homeTeamScore=[]
awayTeamName=[]
awayTeamAbbr=[]
awayTeamCap=[]
awayTeamScore=[]
winner=[]
resultType=[]
tossWinner=[]
tossDecision=[]
bestPlayer=[]
bestPlayerTeam=[]
umpire1=[]
umpire2=[]
tvUmpire=[]
reserveUmpire=[]
matchReferee=[]
inns1Scorecard=[]
inns2Scorecard=[]
teams=[]

for mid in matchid_list:
    url_page='https://hsapi.espncricinfo.com/v1/pages/match/scoreboard?lang=en&leagueId=8048&eventId={match}&liveTest=false&qaTest=false'.format(match=mid)
    driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url_page)
    content = driver.page_source
    soup = BeautifulSoup(content)
    soup_data=json.loads(soup.text)
    
    print('Extracting Match {} data'.format(mid))
    
    match_list.append(mid)
    leagueName.append(soup_data['meta']['leagueName'])
    leagueAbbreviation.append(soup_data['meta']['leagueAbbreviation'])
    season.append(soup_data['header']['matchEvent']['season'])
    matchYear.append(soup_data['meta']['matchYear'])
    matchNum.append(soup_data['meta']['matchNum'])
    date.append(soup_data['content']['about']['matchdays'])
    venue.append(soup_data['header']['matchEvent']['venue']['name'])
    city.append(soup_data['header']['matchEvent']['venue']['city'])
    country.append(soup_data['header']['matchEvent']['venue']['country'])
    homeTeamName.append(soup_data['meta']['homeTeamName'])
    homeTeamAbbr.append(soup_data['meta']['homeTeamAbbr'])
    awayTeamName.append(soup_data['meta']['awayTeamName'])
    awayTeamAbbr.append(soup_data['meta']['awayTeamAbbr'])
    homeTeamCap.append(soup_data['header']['matchEvent']['competitors'][0]['captain']['displayName'])
    awayTeamCap.append(soup_data['header']['matchEvent']['competitors'][1]['captain']['displayName'])
    homeTeamScore.append(soup_data['header']['matchEvent']['competitors'][0]['score'])
    awayTeamScore.append(soup_data['header']['matchEvent']['competitors'][1]['score'])
    
    if str(soup_data['header']['matchEvent']['competitors'][0]['isWinner'])=='True':
        winner.append(soup_data['header']['matchEvent']['competitors'][0]['name'])
    elif str(soup_data['header']['matchEvent']['competitors'][1]['isWinner'])=='True':
        winner.append(soup_data['header']['matchEvent']['competitors'][1]['name'])
    else:
        winner.append('Tie')
        
    resultType.append(soup_data['header']['matchEvent']['statusText'])
    tossWinner.append(soup_data['content']['about']['toss'].split(' , ')[0])
    tossDecision.append(soup_data['content']['about']['toss'].split(' , ')[1])
    bestPlayer.append(soup_data['header']['bestPlayer']['name'])
    bestPlayerTeam.append(soup_data['header']['bestPlayer']['teamName'])
    matchReferee.append(soup_data['content']['about']['referee'][0]['text'])
    umpire1.append(soup_data['content']['about']['umpire'][0]['text'])
    umpire2.append(soup_data['content']['about']['umpire'][1]['text'])
    reserveUmpire.append(soup_data['content']['about']['reserve umpire'][0]['text'])
    tvUmpire.append(soup_data['content']['about']['tv umpire'][0]['text'])
    inns1Scorecard.append(soup_data['content']['innings'][0])
    inns2Scorecard.append(soup_data['content']['innings'][1])
    teams.append(soup_data['content']['teams'][0])
    
    driver.quit()

print(' ')    
print('Scraping process over')

matches_df=pd.DataFrame(list(zip(match_list,leagueName,leagueAbbreviation,season,matchYear,matchNum,date,venue,city,country,
                                 homeTeamName,homeTeamAbbr,homeTeamCap,homeTeamScore,awayTeamName,awayTeamAbbr,awayTeamCap,
                                 awayTeamScore,winner,resultType,tossWinner,tossDecision,bestPlayer,bestPlayerTeam,umpire1,
                                 umpire2,tvUmpire,reserveUmpire,matchReferee,inns1Scorecard,inns2Scorecard,teams)),
                        columns=['Overall Match Id','League Name','League Abbr','Season','Match Year','Match no in Season',
                                 'Date','Venue','City','Country','Home team name','Home team Abbr','Home team cap',
                                 'Home team score','Away team name','Away team Abbr','Away team cap','Away team score',
                                 'Winner','Result type','Toss Winner','Toss decision','MoM','MoM team','Umpire 1','Umpire 2',
                                 'TV Umpire','Reserve Umpire','Match Referee','Inns 1 Scorecard','Inns 2 Scorecard','Team players'])  
matches_df.to_csv('Output folder\Matches_data.csv',index=False)

Extracting Match 1114856 data
 
Scraping process over


### Players data (initial)

In [54]:
players_id=[]
players_df=pd.DataFrame(columns=['Player Id','Full Name','Playing role','Batting style','Bowling style'])

bbb_data=pd.read_csv('Output folder\BBB_data.csv')
#bbb_data.dropna(subset=['Other end bowler id'], inplace=True)
#bbb_data['Other end bowler id']=bbb_data['Other end bowler id'].astype(int)

players_id.append(list(set(bbb_data['Strike batsman id'])))
players_id.append(list(set(bbb_data['Non-strike batsman id'])))
players_id.append(list(set(bbb_data['Main bowler id'])))
players_id.append(list(set(bbb_data['Other end bowler id'])))

players_list = [item for sublist in players_id for item in sublist]
players_list = [x for x in players_list if str(x) != 'nan']
players_list=list(set(players_list))

players_df['Player Id']=players_list
players_df.set_index('Player Id',inplace=True)

for pid in players_df.index.values.tolist():
    url_page='https://www.espncricinfo.com/india/content/player/{}.html'.format(pid)
    driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url_page)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    print('Extracting player {} data'.format(pid))
    
    for tag in soup.findAll('p',attrs={'class':'ciPlayerinformationtxt'}):
        for subtag in tag.findAll('b'):
            if subtag.text == 'Full name':
                players_df.loc[pid]['Full Name']=tag.findAll('span')[0].text
            if subtag.text == 'Playing role':
                players_df.loc[pid]['Playing role']=tag.findAll('span')[0].text
            if subtag.text == 'Batting style':
                players_df.loc[pid]['Batting style']=tag.findAll('span')[0].text
            if subtag.text == 'Bowling style':
                players_df.loc[pid]['Bowling style']=tag.findAll('span')[0].text
    driver.quit()        

print('Scraping process over')    
players_df.to_csv('Output folder\Players_data.csv')

Extracting player 787073 data
Extracting player 4498 data
Extracting player 826901 data
Extracting player 505110 data
Extracting player 297628 data
Extracting player 7069 data
Extracting player 4508 data
Extracting player 270493 data
Extracting player 825211 data
Extracting player 901157 data
Extracting player 6973 data
Extracting player 298438 data
Extracting player 5961 data
Extracting player 398666 data
Extracting player 319439 data
Extracting player 333780 data
Extracting player 240609 data
Extracting player 308967 data
Extracting player 571761 data
Extracting player 8180 data
Extracting player 44149 data
Extracting player 7547 data
Scraping process over


### Get matchids list from Cricsheet list

In [244]:
matchid_list = [f.split('.')[0] for f in os.listdir(r'Input matches')]
matchid_list

['1195618',
 '1195619',
 '1195620',
 '1195621',
 '1195622',
 '1195623',
 '1195624',
 '1195625',
 '1195626',
 '1195627',
 '1195628',
 '1195629',
 '1195630',
 '1195631',
 '1195632',
 '1195633']

### Input matchids manually

In [2]:
matchid_list=[]
print('Enter the matchids for the matches for which ball by ball is required. To stop, please press Enter')
while True:
    text=input()
    if text:
        matchid_list.append(text)
        continue
    else:    
        break
matchid_list        

Enter the matchids for the matches for which ball by ball is required. To stop, please press Enter
1226866
1226863
1226867
1226830



['1226866', '1226863', '1226867', '1226830']

### Matches data (Incremental)

In [3]:
match_list=[]
leagueName=[]
leagueAbbreviation=[]
matchYear=[]
season=[]
matchNum=[]
date=[]
venue=[]
city=[]
country=[]
homeTeamName=[]
homeTeamAbbr=[]
homeTeamCap=[]
homeTeamScore=[]
awayTeamName=[]
awayTeamAbbr=[]
awayTeamCap=[]
awayTeamScore=[]
winner=[]
resultType=[]
tossWinner=[]
tossDecision=[]
bestPlayer=[]
bestPlayerTeam=[]
umpire1=[]
umpire2=[]
tvUmpire=[]
reserveUmpire=[]
matchReferee=[]
inns1Scorecard=[]
inns2Scorecard=[]
teams=[]
matches_data_old=pd.read_csv('Output folder\Matches_data.csv')

for mid in matchid_list:
    url_page='https://hsapi.espncricinfo.com/v1/pages/match/scoreboard?lang=en&leagueId=8048&eventId={match}&liveTest=false&qaTest=false'.format(match=mid)
    driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url_page)
    content = driver.page_source
    soup = BeautifulSoup(content)
    soup_data=json.loads(soup.text)
    
    print('Extracting Match {} data'.format(mid))
    
    match_list.append(mid)
    leagueName.append(soup_data['meta']['leagueName'])
    leagueAbbreviation.append(soup_data['meta']['leagueAbbreviation'])
    season.append(soup_data['header']['matchEvent']['season'])
    matchYear.append(soup_data['meta']['matchYear'])
    matchNum.append(soup_data['meta']['matchNum'])
    date.append(soup_data['content']['about']['matchdays'])
    venue.append(soup_data['header']['matchEvent']['venue']['name'])
    city.append(soup_data['header']['matchEvent']['venue']['city'])
    country.append(soup_data['header']['matchEvent']['venue']['country'])
    homeTeamName.append(soup_data['meta']['homeTeamName'])
    homeTeamAbbr.append(soup_data['meta']['homeTeamAbbr'])
    awayTeamName.append(soup_data['meta']['awayTeamName'])
    awayTeamAbbr.append(soup_data['meta']['awayTeamAbbr'])
    homeTeamCap.append(soup_data['header']['matchEvent']['competitors'][0]['captain']['displayName'])
    awayTeamCap.append(soup_data['header']['matchEvent']['competitors'][1]['captain']['displayName'])
    homeTeamScore.append(soup_data['header']['matchEvent']['competitors'][0]['score'])
    awayTeamScore.append(soup_data['header']['matchEvent']['competitors'][1]['score'])
    
    if str(soup_data['header']['matchEvent']['competitors'][0]['isWinner'])=='True':
        winner.append(soup_data['header']['matchEvent']['competitors'][0]['name'])
    elif str(soup_data['header']['matchEvent']['competitors'][1]['isWinner'])=='True':
        winner.append(soup_data['header']['matchEvent']['competitors'][1]['name'])
    else:
        winner.append('Tie/NR')
        
    resultType.append(soup_data['header']['matchEvent']['statusText'])
    tossWinner.append(soup_data['content']['about']['toss'].split(' , ')[0])
    tossDecision.append(soup_data['content']['about']['toss'].split(' , ')[1])
    if 'bestPlayer' in soup_data['header']:
        bestPlayer.append(soup_data['header']['bestPlayer']['name'])
    else:
        bestPlayer.append(np.nan)
    if 'bestPlayer' in soup_data['header']:
        bestPlayerTeam.append(soup_data['header']['bestPlayer']['teamName'])
    else:
        bestPlayerTeam.append(np.nan)
    matchReferee.append(soup_data['content']['about']['referee'][0]['text'])
    umpire1.append(soup_data['content']['about']['umpire'][0]['text'])
    umpire2.append(soup_data['content']['about']['umpire'][1]['text'])
    
    if 'reserve umpire' in soup_data['content']['about']:
        reserveUmpire.append(soup_data['content']['about']['reserve umpire'][0]['text'])
    else:
        reserveUmpire.append(np.nan)
    
    if 'tv umpire' in soup_data['content']['about']: 
        tvUmpire.append(soup_data['content']['about']['tv umpire'][0]['text'])
    else:
        tvUmpire.append(np.nan)
    
    try:
        inns1Scorecard.append(soup_data['content']['innings'][0])
    except:
        inns1Scorecard.append(np.nan)
    
    try:    
        inns2Scorecard.append(soup_data['content']['innings'][1])
    except:
        inns2Scorecard.append(np.nan)
        
    teams.append(soup_data['content']['teams'][0])
    
    driver.quit()

print(' ')    
print('Scraping process over')

matches_df=pd.DataFrame(list(zip(match_list,leagueName,leagueAbbreviation,season,matchYear,matchNum,date,venue,city,country,
                                 homeTeamName,homeTeamAbbr,homeTeamCap,homeTeamScore,awayTeamName,awayTeamAbbr,awayTeamCap,
                                 awayTeamScore,winner,resultType,tossWinner,tossDecision,bestPlayer,bestPlayerTeam,umpire1,
                                 umpire2,tvUmpire,reserveUmpire,matchReferee,inns1Scorecard,inns2Scorecard,teams)),
                        columns=['Overall Match Id','League Name','League Abbr','Season','Match Year','Match no in Season',
                                 'Date','Venue','City','Country','Home team name','Home team Abbr','Home team cap',
                                 'Home team score','Away team name','Away team Abbr','Away team cap','Away team score',
                                 'Winner','Result type','Toss Winner','Toss decision','MoM','MoM team','Umpire 1','Umpire 2',
                                 'TV Umpire','Reserve Umpire','Match Referee','Inns 1 Scorecard','Inns 2 Scorecard','Team players'])  
matches_df=matches_data_old.append(matches_df,ignore_index=True)
matches_df['Overall Match Id']=matches_df['Overall Match Id'].astype(str)
matches_df=matches_df.sort_values('Overall Match Id')
matches_df.to_csv('Output folder\Matches_data.csv',index=False)

Extracting Match 1226866 data
Extracting Match 1226863 data
Extracting Match 1226867 data
Extracting Match 1226830 data
 
Scraping process over


### Ball by ball data (Incremental)

In [4]:
#Declaring all lists and dataframes
match_list=[]
inns_list=[]
ball_list=[]
over_list=[]
runs_list=[]
comm_short_list=[]
comm_long_list=[]
bat_team_list=[]
bowl_team_list=[]
striker_bat_id_list=[]
nonstriker_bat_id_list=[]
striker_bat_name_list=[]
nonstriker_bat_name_list=[]
main_bow_id_list=[]
other_bow_id_list=[]
main_bow_name_list=[]
other_bow_name_list=[]
isBoundary_list=[]
wide_runs_list=[]
#legbye_runs_list=[]
bye_legbye_runs_list=[]
nb_runs_list=[]
batsman_runs_list=[]
extras_runs_list=[]
is_wkt_list=[]
is_bow_wkt_list=[]
wkt_data_list=[]
curr_bat_list=[]
curr_bow_list=[]
curr_inn_list=[]
match_over_list=[]
ball_data_old=pd.read_csv('Output folder\BBB_data.csv')
match_data=pd.read_csv('Output folder\Matches_data.csv',index_col='Overall Match Id')

for mid in matchid_list:
    #url_match='https://www.espncricinfo.com/series/8048/commentary/{}/trinbago-knight-riders-vs-st-kitts-and-nevis-patriots-1st-match-caribbean-premier-league-2019'.format(mid)
    #driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
    #driver.get(url_match)
    #content = driver.page_source
    #soup = BeautifulSoup(content)
    #team_list=[]
    #for teams in soup.findAll('div',attrs={'class':'teams'}):
     #   for team in teams.findAll('div',attrs={'class':'team'}):
        #team_list.append(team.find('span').attrs['title'])
            #team_list.append(team.findAll('p',attrs={'class':'name'}))
      #      print(team.find('p',attrs={'class':'name'}).text)
    
    #driver.quit()    
    for inns in np.arange(1,3):
        url='https://hsapi.espncricinfo.com/v1/pages/match/comments?lang=en&leagueId=8048&eventId={match}&liveTest=false&period={inns}&page=1'.format(match=mid,inns=inns)
        driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
        driver.get(url)
        content = driver.page_source
        soup = BeautifulSoup(content)
        total_pages=json.loads(soup.text)['pagination']['pageCount']
        driver.quit()
        
        for page in np.arange(total_pages):
            #Code to retrieve json data for the page
            url_page='https://hsapi.espncricinfo.com/v1/pages/match/comments?lang=en&leagueId=8048&eventId={match}&liveTest=false&period={inns}&page={page}'.format(match=mid,inns=inns,page=page+1)
            driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
            driver.get(url_page)
            content = driver.page_source
            soup = BeautifulSoup(content)
            soup_data=json.loads(soup.text)
            print('Extracting_{match}_Inns{inns}_Page{page} content'.format(match=mid,inns=inns,page=page+1))
            #with open('Downloaded webpages\CPL_2019_{match}_Inns{inns}_Page{page}.html'.format(match=mid,inns=inns,page=page+1), 'w') as f:
            #    f.write(content)
            #driver.quit()
            #print('Downloaded CPL_2019_{match}_Inns{inns}_Page{page} webpage'.format(match=mid,inns=inns,page=page+1))
            
            #Code to load page json data into dataframe
            #with open('Downloaded webpages\CPL_2019_{match}_Inns{inns}_Page{page}.html'.format(match=mid,inns=inns,page=page+1)) as fp:
            #    soup_data = json.loads(BeautifulSoup(fp, "html5lib").text)
            for ball_ind in np.arange(len(soup_data['comments'])):
                ball_data=soup_data['comments'][ball_ind]
                if ball_data['ball']==0:
                    continue
                match_list.append(mid)
                inns_list.append(inns)
                if inns==1:
                    bat_team_list.append(match_data.loc[int(mid)]['Home team name'])
                    bowl_team_list.append(match_data.loc[int(mid)]['Away team name'])
                else:
                    bat_team_list.append(match_data.loc[int(mid)]['Away team name'])
                    bowl_team_list.append(match_data.loc[int(mid)]['Home team name'])        
                ball_list.append(ball_data['ball'])
                over_list.append(ball_data['over']+1)
                runs_list.append(ball_data['runs'])
                if 'shortText' in ball_data:
                    comm_short_list.append(ball_data['shortText'])
                else:
                    comm_short_list.append(np.nan)
                comm_long_list.append(ball_data['text'])
                striker_bat_id_list.append(ball_data['currentBatsmen'][0]['id'])
                nonstriker_bat_id_list.append(ball_data['currentBatsmen'][1]['id'])
                striker_bat_name_list.append(ball_data['currentBatsmen'][0]['name'])
                nonstriker_bat_name_list.append(ball_data['currentBatsmen'][1]['name'])
                main_bow_id_list.append(ball_data['currentBowlers'][0]['id'])
                main_bow_name_list.append(ball_data['currentBowlers'][0]['name'])
                extras_runs_list.append(0)

                if 'id' in ball_data['currentBowlers'][1]:
                    other_bow_id_list.append(ball_data['currentBowlers'][1]['id'])
                else:
                    other_bow_id_list.append(np.nan)

                if 'name' in ball_data['currentBowlers'][1]:
                    other_bow_name_list.append(ball_data['currentBowlers'][1]['name'])                    
                else:
                    other_bow_name_list.append(np.nan)

                if str(ball_data['isBoundary'])=='True':
                    isBoundary_list.append(1)
                else:
                    isBoundary_list.append(0)

                if str(ball_data['isWide'])=='True':
                    wide_runs_list.append(ball_data['runs'])
                    nb_runs_list.append(0)
                    bye_legbye_runs_list.append(0)
                    batsman_runs_list.append(0)
                elif ((str(ball_data['isNoball'])=='True')):
                    wide_runs_list.append(0)
                    #bye_legbye_runs_list.append(0)
                    #print(inns,ball_data['over']+1,ball_data['ball'])
                    if (re.search('[(]no ball[)]', ball_data['shortText'])):
                        if ('bye' in ball_data['shortText'].split(', ')[1]):
                            #print('Scenario 1')
                            nb_runs_list.append(1)
                            bye_legbye_runs_list.append(ball_data['runs'] - 1)
                            batsman_runs_list.append(0)
                        else:
                            #print('Scenario 2')
                            nb_runs_list.append(1)
                            bye_legbye_runs_list.append(0)
                            batsman_runs_list.append(ball_data['runs'] - 1)                            
                    else:    
                        nb_runs_list.append(ball_data['runs'])
                        batsman_runs_list.append(0)
                        bye_legbye_runs_list.append(0)
                elif ('bye' in ball_data['shortText'].split(', ')[1]):
                    bye_legbye_runs_list.append(ball_data['runs'])
                    wide_runs_list.append(0)
                    nb_runs_list.append(0)                    
                    batsman_runs_list.append(0)
                else:
                    wide_runs_list.append(0)
                    nb_runs_list.append(0)
                    batsman_runs_list.append(ball_data['runs']) 
                    bye_legbye_runs_list.append(0)

                if 'matchWicket' in ball_data:
                    is_wkt_list.append(1)
                    wkt_data_list.append(ball_data['matchWicket'])
                    if re.search('(run out|obstructing the field)',ball_data['matchWicket']['text']):
                        is_bow_wkt_list.append(0)
                    else:
                        is_bow_wkt_list.append(1)
                else:
                    is_wkt_list.append(0)
                    wkt_data_list.append(0)
                    is_bow_wkt_list.append(0)

                if 'matchOver' in ball_data:
                    match_over_list.append(ball_data['matchOver'])
                else:
                    match_over_list.append(0)

                curr_bat_list.append(ball_data['currentBatsmen'])
                curr_bow_list.append(ball_data['currentBowlers'])
                curr_inn_list.append(ball_data['currentInning'])
            driver.quit()        
    
    print(' ')
    
ball_by_ball_df=pd.DataFrame(list(zip(match_list,inns_list,over_list,ball_list,bat_team_list,bowl_team_list,striker_bat_id_list,striker_bat_name_list,nonstriker_bat_id_list,nonstriker_bat_name_list,main_bow_id_list,main_bow_name_list,other_bow_id_list,other_bow_name_list,isBoundary_list,wide_runs_list,nb_runs_list,bye_legbye_runs_list,extras_runs_list,batsman_runs_list,runs_list,is_wkt_list,is_bow_wkt_list,wkt_data_list,match_over_list,curr_bat_list,curr_bow_list,curr_inn_list,comm_short_list,comm_long_list)),
                             columns=['Match_id','Inns','Over','Ball','Batting team','Bowling team','Strike batsman id','Strike batsman name','Non-strike batsman id','Non-strike batsman name','Main bowler id','Main bowler name','Other end bowler id','Other end bowler name','Is Boundary','Wide runs','No ball runs','Bye and leg bye runs','Extras runs','Batsman runs','Total Runs','Is wicket','Is Bowler wicket','Wicket data','Over data','Batsmen data','Bowlers data','Innings data until then','Short comm','Long Comm'])
ball_by_ball_df['Extras runs']=ball_by_ball_df['Wide runs']+ball_by_ball_df['No ball runs']+ball_by_ball_df['Bye and leg bye runs']
ball_by_ball_df=ball_data_old.append(ball_by_ball_df,ignore_index=True)
ball_by_ball_df=ball_by_ball_df.sort_values(['Match_id','Inns','Over','Ball'])
ball_by_ball_df.to_csv('Output folder\BBB_data.csv',index=False) 

print('Scraping process over')

D:\Downloads\Anaconda\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Extracting_1226866_Inns1_Page1 content
Extracting_1226866_Inns1_Page2 content
Extracting_1226866_Inns1_Page3 content
Extracting_1226866_Inns1_Page4 content
Extracting_1226866_Inns1_Page5 content
Extracting_1226866_Inns2_Page1 content
Extracting_1226866_Inns2_Page2 content
Extracting_1226866_Inns2_Page3 content
Extracting_1226866_Inns2_Page4 content
Extracting_1226866_Inns2_Page5 content
 
Extracting_1226863_Inns1_Page1 content
Extracting_1226863_Inns1_Page2 content
Extracting_1226863_Inns1_Page3 content
Extracting_1226863_Inns1_Page4 content
Extracting_1226863_Inns1_Page5 content
Extracting_1226863_Inns2_Page1 content
Extracting_1226863_Inns2_Page2 content
Extracting_1226863_Inns2_Page3 content
Extracting_1226863_Inns2_Page4 content
Extracting_1226863_Inns2_Page5 content
 
Extracting_1226867_Inns1_Page1 content
Extracting_1226867_Inns1_Page2 content
Extracting_1226867_Inns1_Page3 content
Extracting_1226867_Inns1_Page4 content
Extracting_1226867_Inns1_Page5 content
Extracting_1226867_In

### Players data (incremental)

In [6]:
players_id=[]
players_df_old=pd.read_csv('Output folder\Players_data.csv',index_col='Player Id')
players_df=pd.DataFrame(columns=['Player Id','Full Name','Playing role','Batting style','Bowling style'])

bbb_data=pd.read_csv('Output folder\BBB_data.csv')
#bbb_data.dropna(subset=['Other end bowler id'], inplace=True)
#bbb_data['Other end bowler id']=bbb_data['Other end bowler id'].astype(int)

players_id.append(list(set(bbb_data['Strike batsman id'])))
players_id.append(list(set(bbb_data['Non-strike batsman id'])))
players_id.append(list(set(bbb_data['Main bowler id'])))
players_id.append(list(set(bbb_data['Other end bowler id'])))

players_list = [item for sublist in players_id for item in sublist]
players_list = [x for x in players_list if str(x) != 'nan']
players_list = [x for x in players_list if str(x) != 'undefined']
players_list=[int(p) for p in players_list]
players_list=list(set(players_list))

players_df['Player Id']=[p for p in players_list if p not in players_df_old.index.values.tolist()]
players_df.set_index('Player Id',inplace=True)
print('List of player ids ',players_df.index.values.tolist())

for pid in players_df.index.values.tolist():
    url_page='https://www.espncricinfo.com/india/content/player/{}.html'.format(pid)
    driver = webdriver.Chrome("D:/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url_page)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    print('Extracting player {} data'.format(pid))
    
    for tag in soup.findAll('p',attrs={'class':'ciPlayerinformationtxt'}):
        for subtag in tag.findAll('b'):
            if subtag.text == 'Full name':
                players_df.loc[pid]['Full Name']=tag.findAll('span')[0].text
            if subtag.text == 'Playing role':
                players_df.loc[pid]['Playing role']=tag.findAll('span')[0].text
            if subtag.text == 'Batting style':
                players_df.loc[pid]['Batting style']=tag.findAll('span')[0].text
            if subtag.text == 'Bowling style':
                players_df.loc[pid]['Bowling style']=tag.findAll('span')[0].text
    driver.quit()        

print('Scraping process over')  
players_df=players_df_old.append(players_df)
players_df=players_df.sort_index()
players_df.to_csv('Output folder\Players_data.csv')

List of player ids  [677081, 1182529]
Extracting player 677081 data
Extracting player 1182529 data
Scraping process over


In [5]:
ball_by_ball_df=pd.read_csv('Output folder\BBB_data.csv')
#ball_by_ball_df=ball_by_ball_df.sort_values(['Match_id','Inns','Over','Ball'])
#ball_by_ball_df.to_csv('Output folder\BBB_data.csv',index=False) 
len(set(ball_by_ball_df['Match_id']))

#len(matches_df['Overall Match Id'])

#ball_by_ball_df=ball_by_ball_df.sort_values(['Match id','Inns','Over','Ball'])
#ball_by_ball_df.to_csv('Output folder\BBB_data.csv',index=False) 

#ball_by_ball_df=ball_by_ball_df.sort_values(['Match id','Inns','Over','Ball'])
#ball_by_ball_df.to_csv('Output folder\Test.csv',index=False)

#team_list=[]
#match_data=pd.read_csv('Output folder\Matches_data.csv',index_col='Overall Match Id')
#team_list.append(match_data.loc[int(mid)][['Home team name','Away team name']])
#team_list.shape

#players_df=pd.read_csv('Output folder\Players_data.csv',index_col='Player Id')
#players_df=players_df.sort_values('Player Id')
#players_df=players_df[~players_df.index.duplicated(keep='first')]
#players_df.to_csv('Output folder\Players_data.csv')

#ball_by_ball_df.to_csv('Output folder\BBB_data.csv',index=False) 

173

### Adding Power surge comments

In [3]:
ball_by_ball_df=pd.read_csv('Output folder\BBB_data.csv')
#ball_by_ball_df=ball_by_ball_df.rename(columns={'Match id':'Match_id'})
#ball_by_ball_df['Comments']=np.where((ball_by_ball_df['Match_id'].astype(int)==1226830) & (ball_by_ball_df['Inns']==1) & 
#                                     (ball_by_ball_df['Over'].isin([17,18])),'Power Surge',ball_by_ball_df['Comments'])
#ball_by_ball_df['Comments']=np.where((ball_by_ball_df['Match_id'].astype(int)==1226830) & (ball_by_ball_df['Inns']==2) & 
#                                     (ball_by_ball_df['Over'].isin([14,15])),'Power Surge',ball_by_ball_df['Comments'])
#print(ball_by_ball_df[ball_by_ball_df['Comments']=='Power Surge'])

ball_by_ball_df['Is_legal_ball_team']=np.where((ball_by_ball_df['Wide runs'] > 0) | (ball_by_ball_df['No ball runs'] > 0), 0, 1)
ball_by_ball_df['Is_legal_ball_batsman']=np.where(ball_by_ball_df['Wide runs'] > 0, 0, 1)

ball_by_ball_df.to_csv('Output folder\BBB_data.csv',index=False)

D:\Downloads\Anaconda\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
